# Forecasting in Hindsight: Assessing if Baltimore's 8 Day Forecasted Temperatures are in Line with Historical Norms

### Is the eight day temperature forecast for Baltimore common for this time of year? This notebook seeks to lay the groundwork to answering that question by acquiring and cleaning the data needed to resolve this question.

As such, this notebook is focused on data acquisition and cleaning. It is organized into the following parts:
- <b>API Data Retrieval</b><br>
- <b>API Data Cleanup</b><br>
- <b>Other Data Cleanup</b><br>
- <b>Merge Datasets</b><br>

To begin, we will need to import the necessary tools. These include pandas, json, requests, and datetime.

In [2]:
# import tools / libraries

import pandas as pd
import json
import requests
import datetime

## Using OpenWeatherAPI to retrieve 7 day forecast

Please note that in the API url, you will need to add your own personal API key and enter it where you see '{YOUR API KEY}'. This code cannot run properly without including an API key. To receive your personal API key, you can create an account with OpenWeather at [this link](https://openweathermap.org/guide). Accounts vary in price, but the data retrieved here is able to be retrieved with an unpaid/free account.

The code below will request an 8 day forecast for Baltimore, MD from OpenWeather, create a dataframe from the JSON data using pandas, normalize the data (as the various temperatures are all within a single cell), then save the dataframe to another folder where data is stored. 

If you want to change the location or units of temperature, you can do so by editing the url in the respective locations.

In [34]:
# API URL
url = 'https://api.openweathermap.org/data/2.5/onecall?lat=39.290&lon=-76.612&units=imperial&appid={YOUR API KEY}'

# API Data Retrieval
res = requests.get(url)
resj = res.json()

# Create dataframe from JSON
df_OW1 = pd.DataFrame(resj['daily'])

# normalize data in 'Temp' column
df_OW2 = pd.json_normalize(df_OW1['temp'])

# Concatenate & merge dataframes
# Create object
frames = [df_OW1, df_OW2]
# Merge
df_OW1 = pd.concat(frames, axis = 1, sort = False)

# Save to CSV in Data Folder
df_OW1.to_csv(r'~/Github/DATA601_Project2/Data/Baltimore7DayForecast.csv', index = None)

## API Data Cleanup

Now that we have our csv file, we can pull it and clean it.

In [3]:
# Create dataframe and check info
df1 = pd.read_csv(r'~/Github/DATA601_Project2/Data/Baltimore7DayForecast.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   dt          8 non-null      int64  
 1   sunrise     8 non-null      int64  
 2   sunset      8 non-null      int64  
 3   temp        8 non-null      object 
 4   feels_like  8 non-null      object 
 5   pressure    8 non-null      int64  
 6   humidity    8 non-null      int64  
 7   dew_point   8 non-null      float64
 8   wind_speed  8 non-null      float64
 9   wind_deg    8 non-null      int64  
 10  weather     8 non-null      object 
 11  clouds      8 non-null      int64  
 12  pop         8 non-null      float64
 13  uvi         8 non-null      float64
 14  rain        1 non-null      float64
 15  day         8 non-null      float64
 16  min         8 non-null      float64
 17  max         8 non-null      float64
 18  night       8 non-null      float64
 19  eve         8 non-null      float

We see from above that there are many columns that are unnecessary for analyzing the temperature. As such, we can get rid of these columns. Please note: we are removing the 'temp' column since that column's normalization created a group of columns of different temperature readings.

In [4]:
# Drop unnecessary columns
df1.drop(['sunrise', 'sunset', 'temp', 'feels_like', 'pressure','humidity', 'dew_point', 'wind_speed', 'wind_deg','weather', 'clouds', 'pop', 'uvi', 'rain', 'day', 'night', 'eve', 'morn'], axis = 1, inplace = True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   dt      8 non-null      int64  
 1   min     8 non-null      float64
 2   max     8 non-null      float64
dtypes: float64(2), int64(1)
memory usage: 320.0 bytes


Now that we have gotten rid of superfluous columns, we should work to clean the remaining ones. We notice that column 'dt', which is the date, is listed as an int datatype. It's written in UTC unix code, and should be converted to datetime format. We will also pull that date apart into separate int columns in case we want to work with these more specifically. Additionally, we will add a daily mean temperature of the min and max columns. Finally, we should rearrange the columns in a way that is easier to read.

In [5]:
# Convert UTC unix to datetime
df1['Date'] = pd.to_datetime(df1['dt'],unit='s') # use .dt.date if needed

# Create column for mean temperature
colmean = df1.loc[: , 'min':'max']
df1['mean'] = colmean.mean(axis = 1)

# Create columns for year, month, day
df1['Year'] = df1['Date'].dt.year
df1['Month'] = df1['Date'].dt.month
df1['Day'] = df1['Date'].dt.day

# Drop dt column
df1.drop(['dt'], axis = 1, inplace = True)

# Rearrange columns
df1 = df1[['Date','Year', 'Month','Day', 'min', 'max', 'mean']]

# Check data
df1.head()

,Date,Year,Month,Day,min,max,mean
0,2020-10-03 16:00:00,2020,10,3,53.31,66.31,59.810
1,2020-10-04 16:00:00,2020,10,4,55.31,66.38,60.845
2,2020-10-05 16:00:00,2020,10,5,58.84,69.67,64.255
3,2020-10-06 16:00:00,2020,10,6,57.74,73.74,65.740
4,2020-10-07 16:00:00,2020,10,7,62.28,76.96,69.620


## Pull and Clean Historical Data

Now that we have cleaned the API data, we need to gather and clean the historical data it will be compared to. This historical weather data was taken from Carnegie Mellon University's compiled United States Weather Station data taken from the weather station at Baltimore-Washington Thurgood Marshall International Airport (BWI). These datasets can be found [here](https://kilthub.cmu.edu/articles/dataset/Compiled_daily_temperature_and_precipitation_data_for_the_U_S_cities/7890488?file=20881932). The data for the US Weather Station at BWI is in the file titled 'USW00093721.csv'.

To begin, we will pull that data from our Data folder and view its info.

In [6]:
# Import Baltimore area historical weather data
df2 = pd.read_csv(r'~/Github/DATA601_Project2/Data/USW00093721.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54421 entries, 0 to 54420
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  54421 non-null  int64  
 1   Date        54421 non-null  object 
 2   tmax        53779 non-null  float64
 3   tmin        53795 non-null  float64
 4   prcp        54404 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 2.1+ MB


There's a lot of information to glean from above. First, we see that there are 54,421 rows of data, meaning 54,421 dates are included. We also see that the date is listed as a datatype other than datetime. We will need to fix that. We should also remove the 'prcp' column, as our research question is focused on temperature and not precipitation. To create consistency in column names, we will rename 'tmax' and 'tmin' to 'max' and 'min' respectively. Additionally, we see that there are many null values. We will need to make sure they are not in our final dataframe.

Another important historical detail: the US Weather Station for Baltimore was not in its current location until 1950. It was originally in downtown Baltimore until it was moved to BWI Airport in 1950. Given this inconsistency, we will remove all dates prior to 1950.

The goal of the code below is to address the issues listed above and to make this dataframe identical in layout to the one we created from the API data.

In [7]:
# Convert Date column from object to datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# Create columns for year, month, day
df2['Year'] = df2['Date'].dt.year
df2['Month'] = df2['Date'].dt.month
df2['Day'] = df2['Date'].dt.day

# Drop precipitation column
df2.drop(['prcp'], axis = 1, inplace = True)

# Rename columns
df2.rename(columns = {'tmin':'min', 'tmax':'max'}, inplace = True)

# Create column for mean temperature
colmean1 = df2.loc[: , 'max':'min']
df2['mean'] = colmean1.mean(axis = 1)

# Rearrange columns
df2 = df2[['Date','Year', 'Month','Day', 'min', 'max', 'mean']]

# Remove dates prior to 1950
df2 = df2[df2['Date'] >= '1950-01-01']

# Remove dates outside forecast range
df2 = df2[df2['Month'] == 10]
df2 = df2[df2['Day'] >= 3]
df2 = df2[df2['Day'] <= 10]

# Check to make sure no null values are in dataset
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 560 entries, 29129 to 54338
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    560 non-null    datetime64[ns]
 1   Year    560 non-null    int64         
 2   Month   560 non-null    int64         
 3   Day     560 non-null    int64         
 4   min     560 non-null    float64       
 5   max     560 non-null    float64       
 6   mean    560 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 35.0 KB


## Merge Datasets

We see that there are no nulls in the dataset, and it is structured similarly to our API dataset (df1). We should concatenate the datasets. Additionally, we will add another column related to the date; this column will be focused on the decade the data is from in case we wish to explore and compare data based on this.

Given this included column, we will need to rearrange the columns to make the dataframe easier to read.

In [8]:
# Concatenate dataframes
df3 = pd.concat([df2, df1])

# Create Decade column
# Define an if-else function that will identify and separate decades 
def Decades(row):
    if row['Year'] >= 1950 and row['Year'] <= 1959:
        return '1950s'
    elif row['Year'] >= 1960 and row['Year'] <= 1969:
        return '1960s'
    elif row['Year'] >= 1970 and row['Year'] <= 1979:
        return '1970s'
    elif row['Year'] >= 1980 and row['Year'] <= 1989:
        return '1980s'
    elif row['Year'] >= 1990 and row['Year'] <= 1999:
        return '1990s'
    elif row['Year'] >= 2000 and row['Year'] <= 2009:
        return '2000s'
    elif row['Year'] >= 2010 and row['Year'] <= 2019:
        return '2010s'
    else:
        return '2020'

# incorporate the results of that function into a new column
df3['Decade'] = df3.apply(lambda row: Decades(row), axis = 1)

#Reorganize columns
df3 = df3[['Date', 'Decade', 'Year', 'Month','Day', 'min', 'max', 'mean']]

df3

,Date,Decade,Year,Month,Day,min,max,mean
29129,1950-10-03 00:00:00,1950s,1950,10,3,55.00,84.00,69.500
29130,1950-10-04 00:00:00,1950s,1950,10,4,51.00,65.00,58.000
29131,1950-10-05 00:00:00,1950s,1950,10,5,43.00,63.00,53.000
29132,1950-10-06 00:00:00,1950s,1950,10,6,39.00,65.00,52.000
29133,1950-10-07 00:00:00,1950s,1950,10,7,52.00,68.00,60.000
...,...,...,...,...,...,...,...,...
3,2020-10-06 16:00:00,2020,2020,10,6,57.74,73.74,65.740
4,2020-10-07 16:00:00,2020,2020,10,7,62.28,76.96,69.620
5,2020-10-08 16:00:00,2020,2020,10,8,56.57,65.82,61.195
6,2020-10-09 16:00:00,2020,2020,10,9,49.28,63.70,56.490


## Save the Cleaned Dataset

Now that the datasets have been cleaned, combined, and cleaned again, we can save the dataframe into the Data folder.

In [194]:
df3.to_csv(r'~/Github/DATA601_Project2/Data/CombinedWeather.csv', index = False)

The dataset is ready to be used for data analysis to resolve our research question. This will be done in the other notebook. Please view that one now to see the analysis and findings.